In [1]:
import os
os.chdir("..")

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import gpflow

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm, trange
from gpflow.utilities import positive, to_default_float, set_trainable, print_summary
from gpflow.config import default_float, default_jitter
from gpflow.ci_utils import ci_niter

# Load data, scale, and split

In [2]:
df = pd.read_csv("data/regression/airfoil.csv")
df.head()

,Frequency,AngleAttack,ChordLength,FreeStreamVelocity,SuctionSide,Sound
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [3]:
# Minmax scaling for better network performace
scaler = MinMaxScaler()
D = df.values
D = scaler.fit_transform(D)

# Split in-domain data to data and labels
X, y = D[:,:-1], D[:,-1]

# Split to train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#, random_state=42)



# Helpers for GP model training

In [4]:
def train(m, maxit=500):
    optimizer = tf.optimizers.Adam(learning_rate=0.1)

    with trange(maxit) as t:
        for i in t:
            with tf.GradientTape() as tape:
                tape.watch(m.trainable_variables)
                obj = -m.log_posterior_density()
                grads = tape.gradient(obj, m.trainable_variables)
                optimizer.apply_gradients(zip(grads, m.trainable_variables))

                t.set_description('Iteration %i' % (i+1))
                t.set_postfix(elbo=obj.numpy())

# GP model definition and training

Set up a GP model such that

$$ f(x) \sim \mathcal{GP}(0,\kappa(x,x')), $$
$$ y_i = \prod_{i=1}^n p(y_i \mid f(x_i)), $$

where $\kappa(x,x')$ defines the Gaussian process prior through a covariance function. If you use the ArcCosine covariance function, the model corresponds to (under certain assumptions on the priors on the weights) an infinitely wide single hidden layer feedforward neural network.

Below there are also some other options that can be used for classification and/or alternative model specification.

In [5]:
# The corresponding covariance function
k = gpflow.kernels.ArcCosine(order = 1)
#k.weight_variances.prior = tfp.distributions.Gamma(to_default_float(2), to_default_float(3))
#k.bias_variance.prior = tfp.distributions.Gamma(to_default_float(2), to_default_float(3))
#k.variance.prior = tfp.distributions.Gamma(to_default_float(2), to_default_float(3))
 
# Aleternative model with ARD RBF kernel
#k = gpflow.kernels.RBF(lengthscales=np.ones((1,X_train.shape[1])))    
    
# Create a general GP model (use this for classification)
#m = gpflow.models.VGP((X_train, y_train.reshape(-1,1)), likelihood=gpflow.likelihoods.Bernoulli(), kernel=k)

# Create GP model (this is with a Gaussian likelihood, but use the one below instead)
#m = gpflow.models.VGP((X_train, y_train.reshape(-1,1)), likelihood=gpflow.likelihoods.Gaussian(), kernel=k)

# Create GP regression model (more efficient, leverages conjugacy in the model)
m = gpflow.models.GPR((X_train, y_train.reshape(-1,1)), kernel=k)

# Train GP model
train(m,maxit=100)

2022-06-29 09:03:32.695375: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Iteration 100: 100%|███████████| 100/100 [00:39<00:00,  2.55it/s, elbo=-1.19e+3]


In [6]:
print_summary(m,'notebook')

name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,True,(),float64,4.69891
GPR.kernel.bias_variance,Parameter,Softplus,,True,(),float64,0.150488
GPR.kernel.weight_variances,Parameter,Softplus,,True,(),float64,4.35911
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.00379392


In [7]:
nlpd = tf.reduce_mean(-m.predict_log_density((X_test,y_test.reshape(-1,1))))

print('NLPD %f' % nlpd)

NLPD -1.366751
